# Wrangling 



In [1]:
import pandas as pd
import re
import math
import numpy as np
from datetime import datetime

date = '7_11'
num_scrapes = 3

items_df_1 = pd.concat([pd.read_csv('uniqlo/%s_listings_df_%s0.csv'%(date, str(i))) \
                     for i in range(num_scrapes)])
items_df_2 = pd.read_csv('uniqlo/8_01_listings_df_colab.csv')

In [2]:
'''
    Clean various columns of listings data, cast them into appropriate data types, 
    and concatenate them appropriately
'''
def clean_items_df(items_df):
    items_df = items_df.drop_duplicates().reset_index(drop = True)
    items_df = items_df.iloc[:,2:]
    items_df['num_reviews'] = [int(re.sub("[^0-9]", "", score)) \
            if score != 'BAD' else None for score in items_df['score']]
    items_df['price'] = [float(price[1:].replace(',', '')) for price in items_df['price']]
    items_df['shipping'] = [float(re.sub("[^0-9.]", "", shipping)) if re.sub("[^0-9.]", "", shipping) != '' \
         else 0 for shipping in items_df['shipping']]
    items_df['is_auction_item'] = [num_bids != '0' for num_bids in items_df['num_bids']]
    items_df['num_bids'] = [int(re.sub("[^0-9]", "", num_bids)) for num_bids in items_df['num_bids']]
    items_df['date'] = [datetime.strptime(str(date)[:-4], '%b %d, %Y %H:%M:%S') \
                if str(date)[:-4] != '' else None for date in items_df['date']]
    return items_df


'''
    Adjust and add variables (total price, total caps, total chars, is kaws or not, region) 
    to be included in exploratory data analysis 
'''
def add_variables(items_df):
    items_df['total_price'] = items_df['price'] + items_df['shipping']
    items_df['total_caps'] = [sum([1 for char in title if char.isupper()]) for title in items_df['title']]
    items_df['total_chars'] = [len(title.replace(' ', '')) for title in items_df['title']]
    items_df['is_kaws'] = [title.lower().find('kaws') != -1 for title in items_df['title']]
    items_df['region'] = [loc.split(', ')[1] if loc.split(', ')[-1] == 'United States' else 'International' \
                          for loc in items_df['location'].astype('str')]
    clean_states_dict = dict(zip(['CA', 'MA', 'NY'], ['California', 'Massachusetts', 'New York']))
    items_df['region'] = [clean_states_dict[region] if clean_states_dict.get(region) != None \
                          else region for region in items_df['region']]
    def parse_str(loc_str):
        string = loc_str.split(',')[0]
        final_str = string
        if string in ['United States of America', 'USA Sportswear', 'East Coast', 'Bend', 'USA']:
            final_str = None
        elif string == 'NJ':
            final_str = 'New Jersey'
        elif string == 'miami':
            final_str = 'Florida'
        elif string == 'CA':
            final_str = 'California'
        return final_str
    
    final_strs = [parse_str(loc_str) for loc_str in items_df[items_df.region == 'United States'].location]
    items_df.loc[items_df.region == 'United States', 'region'] = final_strs
    return items_df
    
cl_items_df_1 = add_variables(clean_items_df(items_df_1))
cl_items_df_2 = add_variables(clean_items_df(items_df_2))

In [3]:
'''
    Replace missing time values with estimates such that each unknown listing time
    (or times) must lie at an equal time duration away from its adjacent listings 
    (either known or unknown)
'''
def approx_missing_times(times):
    need_to_approx = False
    new_times = [None] * len(times)
    for i in range(len(times)):
        if not pd.isnull(times[i]):
            #have nulls yet to be estimated & reached non-null time -> fill in previous null values with bounds
            if need_to_approx:
                num_approx = i - maxdex_missing 
                estimates = [times[i] + ((j + 1) * (times[maxdex_date] - times[i]) \
                                           / num_approx) for j in range(num_approx)]
                estimates.reverse()
                new_times[maxdex_missing : i] = estimates
                need_to_approx = False
            maxdex_date = i
        else:
            #found at least one null -> keep track of index and status 
            if not need_to_approx:
                maxdex_missing = i
                need_to_approx = True
    return [times[i] if not pd.isnull(times[i]) else new_times[i] for i in range(len(times))]


cl_items_df_1['date_approx'] = approx_missing_times(cl_items_df_1['date'])
cl_items_df_2['date_approx'] = approx_missing_times(cl_items_df_2['date'])


In [4]:
#trim listings data to only be shirts sold between 5/1/2020 and 8/1/2020
fi_items_1 = cl_items_df_1[(cl_items_df_1['date_approx'] >= datetime(2020, 5, 1)) & \
                        (cl_items_df_1['date_approx'] < datetime(2020, 6, 27))] 
fi_items_2 = cl_items_df_2[(cl_items_df_2['date_approx'] >= datetime(2020, 6, 27)) & \
              (cl_items_df_2['date_approx'] < datetime(2020, 8, 1))] 

cl_items_df = pd.concat([fi_items_1, fi_items_2]).reset_index(drop = True).sort_values(by = 'date_approx')
cl_items_df['mn_day'] = [trans_time.strftime('%-m/%-d') for trans_time in cl_items_df['date_approx']]
cl_items_df['mn_day_date'] = [trans_time.date() for trans_time in cl_items_df['date_approx']]
cl_items_df = cl_items_df[cl_items_df.region != None].reset_index(drop = True)

cl_items_df.to_excel('final_uniqlo_data.xlsx', index = False)